# A simple FM radio App

In this notebook, we will develop a simple app which streams FM audios.

First, let's initialize our IP

In [1]:
import time
import numpy as np
from scipy import signal
from pynq import Overlay, allocate
from rtlsdr import RtlSdr

# 1) init IP

ol = Overlay("../overlay/system-128.bit")

dma = ol.axi_dma_0
hw_fir_1 = ol.fir_complex_0
hw_fir_2 = ol.fir_real_0
hw_discrim = ol.fm_discrim_0

# 2) global parameters

len_in = 1024 * 1500
len_out = int(len_in / 50)

resize = 1 / 50

lpf_b1 = signal.firwin(64, 200e3/(float(2.4e6)/2))
lpf_b2 = signal.firwin(64, 12e3/(float(2.4e6)/10/2))
c1 = np.array(lpf_b1 * resize, dtype=np.float32)
c2 = np.array(lpf_b2 * resize, dtype=np.float32)

# 3) allocate & init buffer

coef_buffer_1 = allocate(shape=(64,), dtype=np.float32)
coef_buffer_2 = allocate(shape=(64,), dtype=np.float32)

input_buffer = allocate(shape=(len_in,), dtype=np.complex64)
output_buffer = allocate(shape=(len_out,), dtype=np.float32)

np.copyto(coef_buffer_1, c1)
np.copyto(coef_buffer_2, c2)

# 4) config physical address

hw_fir_1.register_map.coef_1 = coef_buffer_1.physical_address
hw_fir_2.register_map.coef_1 = coef_buffer_2.physical_address
hw_fir_1.register_map.load_coef = 1
hw_fir_2.register_map.load_coef = 1

def ip_start():
    hw_fir_1.write(0x00, 0x01)
    hw_fir_2.write(0x00, 0x01)
    hw_discrim.write(0x00, 0x01)

Then, let's initialize our RTL-SDR.

In [2]:
sdr = RtlSdr()

fc = 92_700_000    # center frequency
fs = 2_400_000    # sample rate
# 48000
# 50 times

sdr.sample_rate = fs  # Hz
sdr.center_freq = fc  # Hz
sdr.gain = 4

Found Rafael Micro R820T/2 tuner


Optionally, you can choose to increase usb memory buffer so that larger samples can be received at a time.

In [3]:
# increase usb memory buffer
!echo 0 > /sys/module/usbcore/parameters/usbfs_memory_mb

Now, let's utilize `ipywidgets` and `asyncio` to stream the FM audio

In [9]:
from IPython.display import Audio
import ipywidgets as widgets
import nest_asyncio
import asyncio
from jupyter_ui_poll import ui_events

nest_asyncio.apply()

flag = 0

it_num = 10

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=90_000_000,    # Minimum value
    max=100_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal',  # Orientation of the slider
)

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=len_in*2, format='bytes', loop=None):        
        # do something with samples
        global flag

        # convert received data to array
        arr = np.ctypeslib.as_array(data)

        # convert the data type of the array to complex128
        samples = arr.astype(np.float64).view(np.complex128)

        # copy the samples to the input buffer
        np.copyto(input_buffer, samples)

        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)

        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer, autoplay=True, rate=48000, normalize=False))
        
        # poll ui events
        with ui_events() as poll:
            poll(1)
            sdr.center_freq = slider.value
        
        it = it + 1
        print("sample %d" % it)
        if it == it_num:
            break
    
display(slider)

asyncio.run(streaming())

IntSlider(value=92700000, description='Center Freq:', max=100000000, min=90000000, step=100000)

sample 1


sample 2


sample 3


sample 4


...

sample 6


sample 7


sample 8


sample 9


sample 10


You can hide the progress bar by the following code.

In [7]:
from IPython.display import HTML, display
# hide the play bar from IPython.display.Audio
display_audio_css = """
<style>
audio { display: none }
</style>
"""
# unhide: { display: block }
display(HTML(display_audio_css))

Next, we can test the CPU time of each operation.

In [8]:
from IPython.display import Audio
import ipywidgets as widgets
import nest_asyncio
import asyncio
from jupyter_ui_poll import ui_events

nest_asyncio.apply()

flag = 0

it_num = 10

slider = widgets.IntSlider(
    value=92_700_000,  # Initial value
    min=90_000_000,    # Minimum value
    max=100_000_000,  # Maximum value
    step=100_000,   # Step size
    description='Center Freq:',  # Description displayed next to the slider
    orientation='horizontal',  # Orientation of the slider
)

len_output_buffer = len(output_buffer)

async def streaming():
    it = 0
    async for data in sdr.stream(num_samples_or_bytes=1024 * 3000, format='bytes', loop=None):        
        # do something with samples
        global flag
        global etg, stg

        st1 = time.time()
        # convert received data to array
        arr = np.ctypeslib.as_array(data)
        et1 = time.time()
        print("cpu - as_array:\t %f s" % (et1 - st1))
        
        st2 = time.time()
        # convert the data type of the array to complex128
        samples = arr.astype(np.float64)
        et2 = time.time()
        print("cpu - astype:\t %f s" % (et2 - st2))
        
        st3 = time.time()
        samples = samples.view(np.complex128)
        et3 = time.time()
        print("cpu - view:\t %f s" % (et3 - st3))
        
        st4 = time.time()
        # copy the samples to the input buffer
        np.copyto(input_buffer, samples)
        et4 = time.time()
        print("cpu - copy:\t %f s" % (et4 - st4))
        
        # make sure the dma is idle
        if flag == 1:
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        else:
            flag = 1

        # start fir and discrim ip
        ip_start()
        
        # send to DMA
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        
        st5 = time.time()
        # poll ui events
        with ui_events() as poll:
            poll(1)
            sdr.center_freq = slider.value
        et5 = time.time()
        print("cpu - poll ui:\t %f s" % (et5 - st5))

        etg = time.time()
        print("total cpu time:\t %f s" % (et5 - st5 + et4 - st4 + et3 - st3 + et2 - st2 + et1 - st1))
        print("interval between play: %f s" % (etg - stg))
        # play the audio!
        # make sure to disable automatic normalization
        # and manually normalize the array to [-1, 1]
        display(Audio(output_buffer, autoplay=True, rate=48000, normalize=False))
        print("actual played time: %f s" % (len_output_buffer/48000))
        print("==========================")
        
        stg = time.time()
        
        it = it + 1
        if it == it_num:
            break
    
display(slider)
stg = time.time()
asyncio.run(streaming())

IntSlider(value=92700000, description='Center Freq:', max=100000000, min=90000000, step=100000)

cpu - as_array:	 0.000063 s
cpu - astype:	 0.447046 s
cpu - view:	 0.000052 s
cpu - copy:	 0.093544 s
cpu - poll ui:	 0.050530 s
total cpu time:	 0.591235 s
interval between play: 0.967090 s


actual played time: 0.640000 s
cpu - as_array:	 0.000074 s
cpu - astype:	 0.472779 s
cpu - view:	 0.000047 s
cpu - copy:	 0.093400 s
cpu - poll ui:	 0.049443 s
total cpu time:	 0.615743 s
interval between play: 0.628259 s


actual played time: 0.640000 s
cpu - as_array:	 0.000071 s
cpu - astype:	 0.468479 s
cpu - view:	 0.000055 s
cpu - copy:	 0.093199 s
cpu - poll ui:	 0.049469 s
total cpu time:	 0.611273 s
interval between play: 0.616280 s


actual played time: 0.640000 s
cpu - as_array:	 0.000071 s
cpu - astype:	 0.476362 s
cpu - view:	 0.000053 s
cpu - copy:	 0.093338 s
cpu - poll ui:	 0.049284 s
total cpu time:	 0.619108 s
interval between play: 0.624190 s


...

actual played time: 0.640000 s
cpu - as_array:	 0.000077 s
cpu - astype:	 0.472168 s
cpu - view:	 0.000051 s
cpu - copy:	 0.093856 s
cpu - poll ui:	 0.049282 s
total cpu time:	 0.615435 s
interval between play: 0.619680 s


actual played time: 0.640000 s
cpu - as_array:	 0.000287 s
cpu - astype:	 0.472490 s
cpu - view:	 0.000050 s
cpu - copy:	 0.089913 s
cpu - poll ui:	 0.049355 s
total cpu time:	 0.612096 s
interval between play: 0.619944 s


actual played time: 0.640000 s
cpu - as_array:	 0.000071 s
cpu - astype:	 0.468584 s
cpu - view:	 0.000047 s
cpu - copy:	 0.089837 s
cpu - poll ui:	 0.049094 s
total cpu time:	 0.607632 s
interval between play: 0.614293 s


actual played time: 0.640000 s
cpu - as_array:	 0.000074 s
cpu - astype:	 0.474652 s
cpu - view:	 0.000051 s
cpu - copy:	 0.094352 s
cpu - poll ui:	 0.049335 s
total cpu time:	 0.618462 s
interval between play: 0.623452 s


actual played time: 0.640000 s


As shown above, the `astype` operation has consumed the most of the cpu time. We can also offload this operation to the FPGA.